In [86]:
import pandas as pd
import plotly as py
import numpy as np
py.tools.set_credentials_file(username='clcarver', api_key='IsOMdkWRhw3WZouQQO9S')

In [184]:
# Load in codes: df_codes
df_codes = pd.read_csv('../country_codes.csv')

In [185]:
# Load in data: df
df_clean = pd.read_csv('https://s3.amazonaws.com/clcarverloans/data/df_cleaned.csv')

In [186]:
default_by_country = pd.DataFrame(df_clean.groupby('Country Code').sum()['Delinquent']/df_clean.groupby('Country Code').count()['Delinquent']).sort_values('Delinquent')

In [188]:
# Add in 3 digit codes and add missing countries
df_merged = default_by_country.merge(df_codes, how='outer', left_index=True, right_on='Code_2')

In [189]:
df_merged['Delinquent'].fillna(0, inplace=True)

In [190]:
df_merged['Delinquent'] = round((df_merged['Delinquent']*100), 2)

In [198]:
data = [ dict(
        type = 'choropleth',
        locations = df_merged['Code_3'],
        z = df_merged['Delinquent'],
        text = df_merged['Country'],
        colorscale = [[0,"rgb(204,208,210)"],[0.01,"rgb(139,194,224)"],[0.15,"rgb(66,134,244)"],\
            [0.25,"rgb(87,132,196)"],[0.45,"rgb(86,130,151)"],[1,"rgb(38,61,86)"]],
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = True,
            ticksuffix = '%',
            title = 'Default Rates'),
      ) ]

layout = dict(
    title = '2010 Microfinance Default Rates',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.plotly.iplot( fig, validate=False, filename='default_rate_by_country' )